<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><span><a href="#Text-Classification" data-toc-modified-id="Text-Classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Text Classification</a></span><ul class="toc-item"><li><span><a href="#Subjectivity-Dataset" data-toc-modified-id="Subjectivity-Dataset-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Subjectivity Dataset</a></span></li><li><span><a href="#Tokenization" data-toc-modified-id="Tokenization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Tokenization</a></span><ul class="toc-item"><li><span><a href="#Simple-Tokenization" data-toc-modified-id="Simple-Tokenization-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Simple Tokenization</a></span></li><li><span><a href="#Much-better-tokenization-with-Spacy" data-toc-modified-id="Much-better-tokenization-with-Spacy-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Much better tokenization with Spacy</a></span></li></ul></li><li><span><a href="#Split-dataset-in-train-and-validation" data-toc-modified-id="Split-dataset-in-train-and-validation-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Split dataset in train and validation</a></span></li><li><span><a href="#Word-to-index-mapping" data-toc-modified-id="Word-to-index-mapping-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Word to index mapping</a></span></li><li><span><a href="#Sentence-encoding" data-toc-modified-id="Sentence-encoding-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Sentence encoding</a></span></li><li><span><a href="#Embedding-layer" data-toc-modified-id="Embedding-layer-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Embedding layer</a></span></li><li><span><a href="#Continuous-Bag-of-Words-Model" data-toc-modified-id="Continuous-Bag-of-Words-Model-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Continuous Bag of Words Model</a></span></li></ul></li><li><span><a href="#Training-the-CBOW-model" data-toc-modified-id="Training-the-CBOW-model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Training the CBOW model</a></span></li><li><span><a href="#Data-loaders-for-SGD" data-toc-modified-id="Data-loaders-for-SGD-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Data loaders for SGD</a></span></li></ul></div>

In [76]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

# Text Classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
unpack_dataset()

--2021-04-21 20:59:59--  http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
Resolving www.cs.cornell.edu (www.cs.cornell.edu)... 132.236.207.36
Connecting to www.cs.cornell.edu (www.cs.cornell.edu)|132.236.207.36|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519599 (507K) [application/x-gzip]
Saving to: ‘rotten_imdb.tar.gz’

rotten_imdb.tar.gz  100%[===================>] 507.42K  1.31MB/s    in 0.4s    

2021-04-21 20:59:59 (1.31 MB/s) - ‘rotten_imdb.tar.gz’ saved [519599/519599]

mkdir: data: File exists
x quote.tok.gt9.5000
x plot.tok.gt9.5000
x subjdata.README.1.0


In [4]:
!ls data

Readme.txt          hour.csv            names_train.csv     quote.tok.gt9.5000
day.csv             names_test.csv      plot.tok.gt9.5000   subjdata.README.1.0


In [77]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [78]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/names_train.csv'),
 PosixPath('data/names_test.csv'),
 PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000'),
 PosixPath('data/hour.csv'),
 PosixPath('data/Readme.txt'),
 PosixPath('data/day.csv')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [79]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [8]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [9]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [10]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Better tokenization with Spacy

In [11]:
#!pip install -U spacy

In [14]:
import spacy

In [17]:
# first time run this
#!python3 -m spacy download en_core_web_sm

In [19]:
tok = spacy.load('en_core_web_sm')

In [20]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [21]:
len(obj_lines)

5000

In [22]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [23]:
test = tok(obj_lines[0])

In [24]:
np.array([x for x in test])

array([the, movie, begins, in, the, past, where, a, young, boy, named,
       sam, attempts, to, save, celebi, from, a, hunter, ., 
], dtype=object)

## Split dataset in train and validation

In [80]:
from sklearn.model_selection import train_test_split

In [81]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [82]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [83]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'),
 array([1., 0., 0., 1., 1.]))

## Word to index mapping
In interest of time we will tokenize without spaCy. Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [85]:
from collections import defaultdict

In [86]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [87]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [89]:
#word_count

In [90]:
len(word_count.keys())

21415

In [91]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [92]:
len(word_count.keys())

4065

In [93]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [95]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [39]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_val_len = np.array([len(x.split()) for x in X_val])

In [96]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

40.0

In [97]:
X_train[0]

'will god let her fall or give her a new path ?'

In [98]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

9

In [99]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([11,  3,  5,  7,  4, 12,  6,  7,  8, 10,  2,  9])

In [100]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc

In [101]:
encode_sentence(X_train[0])

array([11,  3,  5,  7,  4, 12,  6,  7,  8, 10,  2,  9,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0], dtype=int32)

In [102]:
x_train_len = np.minimum(x_train_len, 40)
x_val_len = np.minimum(x_val_len, 40)

In [47]:
x_train = np.vstack([encode_sentence(x) for x in X_train])
x_train.shape

(8000, 40)

In [48]:
x_val = np.vstack([encode_sentence(x) for x in X_val])
x_val.shape

(2000, 40)

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [49]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.7581, -0.6954,  1.3289,  0.7559],
        [ 1.2147,  1.3281,  1.9152,  0.1689],
        [-0.1969,  0.7253, -0.3557, -0.3875],
        [-0.0332,  0.8995,  0.6054,  0.2052],
        [-1.3950, -0.7561, -0.3994,  0.0034],
        [ 1.0961,  0.8007,  0.6833,  1.4554],
        [-0.7350,  0.3209, -0.5843, -1.2058],
        [ 1.1652,  0.6358,  0.5472, -0.2447],
        [ 1.4607, -0.4960, -0.6349,  0.4898]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [50]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[-0.7581, -0.6954,  1.3289,  0.7559],
         [-0.0332,  0.8995,  0.6054,  0.2052],
         [-0.7581, -0.6954,  1.3289,  0.7559],
         [-1.3950, -0.7561, -0.3994,  0.0034],
         [-0.7581, -0.6954,  1.3289,  0.7559],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [51]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [52]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [104]:
embed(a).shape

torch.Size([2, 3, 4])

In [103]:
embed(a)

tensor([[[-0.7581, -0.6954,  1.3289,  0.7559],
         [-0.0332,  0.8995,  0.6054,  0.2052],
         [-0.7581, -0.6954,  1.3289,  0.7559]],

        [[-0.7581, -0.6954,  1.3289,  0.7559],
         [-0.1969,  0.7253, -0.3557, -0.3875],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [54]:
embed(a).sum(dim=1)

tensor([[-1.5494, -0.4912,  3.2632,  1.7171],
        [-0.9550,  0.0299,  0.9732,  0.3684]], grad_fn=<SumBackward1>)

In [55]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[-0.5165, -0.1637,  1.0877,  0.5724],
        [-0.4775,  0.0150,  0.4866,  0.1842]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [106]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s
        x = self.linear(x)
        return x

In [107]:
model = CBOW(vocab_size=5, emb_size=3)

In [108]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.3654, -1.5260,  0.6905],
        [-0.5017, -0.6429, -0.4537],
        [-1.3262,  1.0019,  0.0299],
        [-0.5894,  0.4454,  1.5460]], requires_grad=True)

In [109]:
s = s.view(s.shape[0], 1)
model(a, s)

RuntimeError: The size of tensor a (2) must match the size of tensor b (5) at non-singleton dimension 0

# Training the CBOW model 

In [116]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [117]:
def val_metrics(model):
    model.eval()
    x = torch.LongTensor(x_val) #.cuda()
    y = torch.Tensor(y_val).unsqueeze(1) #).cuda()
    s = torch.Tensor(x_val_len).view(x_val_len.shape[0], 1)
    y_hat = model(x, s)
    loss = F.binary_cross_entropy_with_logits(y_hat, y)
    y_pred = y_hat > 0
    correct = (y_pred.float() == y).float().sum()
    accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [118]:
# accuracy of a random model should be around 0.5
val_metrics(model)

(0.7025092840194702, 0.47200000286102295)

In [119]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        model.train()
        x = torch.LongTensor(x_train)  #.cuda()
        y = torch.Tensor(y_train).unsqueeze(1)
        s = torch.Tensor(x_train_len).view(x_train_len.shape[0], 1)
        y_hat = model(x, s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        val_loss, val_accuracy = val_metrics(model)
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (loss.item(), val_loss, val_accuracy))

In [124]:
model = CBOW(vocab_size=V, emb_size=50)
train_epocs(model, epochs=10, lr=0.1)

train_loss 0.707 val_loss 0.667 val_accuracy 0.538
train_loss 0.665 val_loss 0.574 val_accuracy 0.793
train_loss 0.565 val_loss 0.491 val_accuracy 0.836
train_loss 0.471 val_loss 0.402 val_accuracy 0.862
train_loss 0.371 val_loss 0.322 val_accuracy 0.890
train_loss 0.281 val_loss 0.281 val_accuracy 0.896
train_loss 0.228 val_loss 0.255 val_accuracy 0.903
train_loss 0.187 val_loss 0.249 val_accuracy 0.899
train_loss 0.160 val_loss 0.257 val_accuracy 0.898
train_loss 0.143 val_loss 0.262 val_accuracy 0.904


In [125]:
train_epocs(model, epochs=10, lr=0.01)

train_loss 0.124 val_loss 0.259 val_accuracy 0.907
train_loss 0.119 val_loss 0.258 val_accuracy 0.909
train_loss 0.116 val_loss 0.258 val_accuracy 0.910
train_loss 0.113 val_loss 0.257 val_accuracy 0.908
train_loss 0.110 val_loss 0.257 val_accuracy 0.909
train_loss 0.106 val_loss 0.257 val_accuracy 0.909
train_loss 0.103 val_loss 0.258 val_accuracy 0.909
train_loss 0.100 val_loss 0.258 val_accuracy 0.910
train_loss 0.098 val_loss 0.259 val_accuracy 0.911
train_loss 0.095 val_loss 0.259 val_accuracy 0.911


# Data loaders for SGD

Nearly all of deep learning is powered by one very important algorithm: **stochastic gradient descent (SGD)**. SGD can be seeing as an approximation of **gradient descent** (GD). In GD you have to run through *all* the samples in your training set to do a single itaration. In SGD you use *only one* or *a subset*  of training samples to do the update for a parameter in a particular iteration. The subset use in every iteration is called a **batch** or **minibatch**.

In [66]:
from torch.utils.data import Dataset, DataLoader

Next we are going to create a data loader. The data loader provides the following features:
* Batching the data
* Shuffling the data
* Load the data in parallel using multiprocessing workers.

In [67]:
def encode_sentence2(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [68]:
encode_sentence2(X_train[0])

(array([11,  3,  5,  7,  4, 12,  6,  7,  8, 10,  2,  9,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32),
 12)

In [126]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence2(x)
        return x, self.y[idx], s
    
sub_dataset_train = SubjectivityDataset(X_train, y_train)

In [127]:
train_loader = DataLoader(sub_dataset_train, batch_size=5, shuffle=True)
x, y, s = next(iter(train_loader))

In [128]:
x, y, s

(tensor([[  19,   19,   19, 3628, 1207,   14,  681,    1,   52,  159, 1167,   18,
           594,    1,   37, 3218,    8,  113,    1,  131,   52,  117,   18, 2633,
            19,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [ 952, 3801,    1,   26,   14,  149,   72,  415,    8,    1,   66,   14,
          2087,    1,  109,   98,  211,    1,  175,   37,  931,   61,   38,   88,
            14,  373,   19,   30,  245,   33,   26,   14,  239,    1,    1, 3837,
            88,    1, 1077,  973],
         [  81,    1,   26,    1,   72,  496,  175,   14,    1,   26,   30,   36,
            14,   97,   52,  113, 1575, 3576,   19,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0],
         [  81,   14,    1,   26,   14, 1640, 1952,   26,  176,   36,   14,    1,
            26, 3105,  196,   37, 1929,   14,  611, 1952,   88,    8, 3106,

In [129]:
model = CBOW(vocab_size=V, emb_size=50)

In [130]:
train_loader = DataLoader(sub_dataset_train, batch_size=500, shuffle=True)

In [131]:
def train_epocs(model, epochs=10, lr=0.01):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for i in range(epochs):
        total_loss = 0
        total = 0
        model.train()
        for x, y, s in train_loader:
            x = x.type(torch.LongTensor)  #.cuda()
            y = y.type(torch.FloatTensor).unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x, s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += x.size(0)*loss.item()
            total += x.size(0)
        train_loss = total_loss/total
        val_loss, val_accuracy = val_metrics(model)
        
        print("train_loss %.3f val_loss %.3f val_accuracy %.3f" % (train_loss, val_loss, val_accuracy))

In [75]:
train_epocs(model, epochs=10)

train_loss 0.672 val_loss 0.626 val_accuracy 0.770
train_loss 0.548 val_loss 0.461 val_accuracy 0.859
train_loss 0.368 val_loss 0.318 val_accuracy 0.890
train_loss 0.250 val_loss 0.259 val_accuracy 0.905
train_loss 0.191 val_loss 0.237 val_accuracy 0.906
train_loss 0.157 val_loss 0.232 val_accuracy 0.907
train_loss 0.132 val_loss 0.230 val_accuracy 0.910
train_loss 0.113 val_loss 0.234 val_accuracy 0.908
train_loss 0.098 val_loss 0.237 val_accuracy 0.907
train_loss 0.085 val_loss 0.244 val_accuracy 0.902
